In [1]:
import os
import json
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv("RUNPOD_TOKEN"),
    base_url=os.getenv("RUNPOD_CHATBOT_URL"),
)
model_name = os.getenv("MODEL_NAME")

In [3]:
def get_chatbot_response(client, model_name, messages, temperature=0):  # FIXED
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})

    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=0.0,
        max_tokens=2000,
        top_p=0.8,
    ).choices[0].message.content
    return response

In [4]:
import re

def classify_query(query):
    query = query.lower()
    if "instagram" in query:
        app = "instagram"
    elif "whatsapp" in query:
        app = "whatsapp"
    else:
        app = None

    if "notification" in query:
        metric = "notifications"
    elif "time" in query and "open" in query:
        metric = "times_opened"
    elif "usage" in query or "used" in query:
        metric = "usage"
    else:
        metric = None

    return app, metric


In [5]:
import json

def load_forecast_data(app, metric):
    file_map = {
        ("instagram", "usage"): "instagram_usage_forecast_filtered.json",
        ("instagram", "notifications"): "instagram_notifications_forecast_filtered.json",
        ("instagram", "times_opened"): "instagram_times_opened_forecast_filtered.json",
        ("whatsapp", "usage"): "whatsapp_usage_forecast_filtered.json",
        ("whatsapp", "notifications"): "whatsapp_notifications_forecast_filtered.json",
        ("whatsapp", "times_opened"): "whatsapp_times_opened_forecast_filtered.json",
    }
    key = (app, metric)
    if key not in file_map:
        return None

    path = os.path.join("json_data_filtered", file_map[key])
    with open(path, 'r') as f:
        data = json.load(f)
    return data

In [6]:
def format_forecast_summary(data, max_days=5):
    context = []
    for entry in data[-max_days:]:
        date = entry["ds"]
        trend = round(entry["trend"], 2)
        pred = round(entry["yhat"], 2)
        context.append(f"On {date}, predicted value is {pred} with trend {trend}.")
    return "\n".join(context)

In [16]:
def answer_user_query(query):
    app, metric = classify_query(query)
    if not app or not metric:
        return "Sorry, I couldn't detect which app or metric you're referring to."

    data = load_forecast_data(app, metric)
    if not data:
        return f"No data found for {app} - {metric}."

    context = format_forecast_summary(data)

    prompt = (
        f"You're an assistant analyzing forecasted mobile app usage.\n"
        f"Treat the current date as **2022-09-21**, and note that the forecast extends up to **2022-10-21**.\n\n"
        f"If the user asks for data of any particular data of past or future check the files and then give it to him if it exists"
        f"App: {app.capitalize()}\nMetric: {metric.replace('_', ' ').capitalize()}\n"
        f"Here is the forecast summary for recent days:\n"
        f"{context}\n\n"
        f"User Question: {query}\n"
        f"Answer the user's question in natural language, with reasoning if needed."
    )

#     prompt = (
#     f"You're an assistant analyzing forecasted mobile app usage behavior.\n\n"
#     f"The forecast data is for the mobile app: **{app.capitalize()}**, and the metric: **{metric.replace('_', ' ').capitalize()}**.\n"
#     f"You are given historical trends and forecasted values between past and future dates.\n"
#     f"Treat the current date as **2022-09-21**, and note that the forecast extends up to **2022-10-21**.\n\n"
#     f"Here is the forecast summary for recent days:\n"
#     f"{context}\n\n"
#     f"User Question: {query}\n\n"
#     f"Based on the forecasted trends, provide a helpful, natural language answer with reasoning if needed. "
#     f"Summarize patterns, highlight expected increases or decreases, and be clear about what the data shows."
# )


    messages = [  # FIXED
        {'role': 'system', 'content': prompt},
        {'role': 'user', 'content': query}
    ]

    return get_chatbot_response(client, model_name, messages)

In [8]:
print(answer_user_query("How much is my WhatsApp usage expected to increase this week?"))

Based on the forecasted values, it appears that your WhatsApp usage is trending upwards. The trend values for each day are increasing, indicating a steady growth in usage.

To estimate the increase in usage this week, let's look at the trend values. The trend value for the first day (2022-10-17) is 137.96, and the trend value for the last day (2022-10-21) is 142.58. This represents an increase of 4.62 (142.58 - 137.96) over the course of 4 days.

To calculate the expected increase in usage this week, we can multiply this daily increase by 7 (the number of days in a week): 4.62 * 7 = 32.34.

So, based on this analysis, your WhatsApp usage is expected to increase by approximately 32.34 units this week.


In [9]:
print(answer_user_query("Do I get more Instagram notifications on weekends?"))

Based on the forecasted data, it doesn't seem like there's a clear pattern of increased notifications on weekends. In fact, the trend is generally downward, indicating a decrease in notifications over the days.

On 2022-10-17, the predicted value is -15.4, which is a significant decrease. The trend for this day is -9.13, which is also negative.

On 2022-10-18, the predicted value is -9.37, which is still a decrease. The trend for this day is -10.09, which is also negative.

On 2022-10-19, the predicted value is -19.12, which is an even larger decrease. The trend for this day is -11.04, which is also negative.

On 2022-10-20, the predicted value is -11.28, which is still a decrease. The trend for this day is -11.99, which is also negative.

On 2022-10-21, the predicted value is 1.49, which is a small increase. However, the trend for this day is -12.94, which is still negative.

There doesn't seem to be a clear pattern of increased notifications on weekends, as the trend is generally dow

In [18]:

print(answer_user_query("How many notifications on WhatsApp did i receive 2022-10-20"))

According to the forecast summary, the predicted value for notifications on WhatsApp on 2022-10-20 is 220.15. This is the forecasted number of notifications you would have received on that day.
